# Analysis of Experiments

## Read data

Look for data in each subdirectory of `OUTPUD_DIR`

In [1]:
from pathlib import Path

import pandas as pd

In [2]:
OUTPUT_DIR = Path('output')


DATA_DIRS: list[Path] = [p for p in OUTPUT_DIR.iterdir() if p.is_dir()]

In [3]:
import json


RAW_DATA: list[dict] = []

for data_dir in DATA_DIRS:
    data = []
    for rfile in data_dir.glob('*_results.json'):
        with rfile.open() as fp:
            result = json.load(fp)
            result['src'] = data_dir.name
            data.append(result)
    if len(data) < 1 and data_dir.joinpath('results.json').exists():
        with data_dir.joinpath('results.json').open() as fp:
            results = json.load(fp)
            for result in results:
                result['src'] = data_dir.name
                data.append(result)

    RAW_DATA.extend(data)

RAW_DATA_df = pd.json_normalize(RAW_DATA)

RAW_DATA_df.sample(5)

,id,generator,cases,src,control_flow.levenshtein,control_flow.hamming,params.model,params.traces,params.events,params.noise,...,stats.times.total,stats.times.cpu,stats.times.solve,stats.times.unsat,stats.times.sat,stats.timeout,stats.timedout,stats.models,data_flow.levenshtein,data_flow.hamming
267,levenshtein_caise2025_experimental_model_0100_...,PBLogGeneratorLevenshtein,1,caise2025_tabu_2024-12-06T08:21+00:00,NaN,NaN,caise2025_experimental_model,100,70,0,...,160.192808,160.227960,120.041057,0.0,0.000000,120,True,2,NaN,NaN
125,levenshtein_caise2025_experimental_model_0100_...,PBLogGeneratorLevenshtein,7,caise2025_tabu_2024-12-06T08:21+00:00,0.316667,0.373810,caise2025_experimental_model,100,40,0,...,232.179382,231.079103,192.396823,0.0,71.579907,120,True,8,0.373810,0.373810
628,hamming_caise2025_experimental_model_4000_070_...,PBLogGeneratorHamming,3,caise2025_2024-12-05T12:36+00:00,0.300000,0.409524,caise2025_experimental_model,4000,70,0,...,286.933052,286.980601,282.819161,0.0,42.686224,120,True,5,0.747619,0.761905
444,hamming_caise2025_experimental_model_0500_020_...,PBLogGeneratorHamming,7,caise2025_2024-12-05T12:36+00:00,0.309524,0.323810,caise2025_experimental_model,500,20,0,...,479.864117,479.962252,478.946106,0.0,238.895688,120,True,9,0.671429,0.671429
735,baseline_caise2025_experimental_model_0100_020_00,PBLogGeneratorBaseline,100,caise2025_base_2024-12-06T08:20+00:00,0.000000,0.000000,caise2025_experimental_model,100,20,0,...,0.000000,0.000000,0.000000,0.0,0.000000,120,False,100,0.053364,0.053364


In [4]:
RAW_DATA_df.columns

Index(['id', 'generator', 'cases', 'src', 'control_flow.levenshtein',
       'control_flow.hamming', 'params.model', 'params.traces',
       'params.events', 'params.noise', 'params.threshold', 'params.randomise',
       'stats.times.total', 'stats.times.cpu', 'stats.times.solve',
       'stats.times.unsat', 'stats.times.sat', 'stats.timeout',
       'stats.timedout', 'stats.models', 'data_flow.levenshtein',
       'data_flow.hamming'],
      dtype='object')

### Summarise data

In [5]:
SUMMARY = RAW_DATA_df[['src', 'id', 'generator']].copy()

SUMMARY['threshold'] = RAW_DATA_df['params.threshold']
SUMMARY['randomised'] = ((RAW_DATA_df['generator'] == 'PBLogGeneratorRandom') | RAW_DATA_df['params.randomise'])
SUMMARY['model'] = RAW_DATA_df['params.model']
SUMMARY['case_size'] = RAW_DATA_df['params.events']
SUMMARY['log_size'] = RAW_DATA_df['params.traces']
SUMMARY['generated'] = RAW_DATA_df['cases']
SUMMARY['coverage'] = RAW_DATA_df['cases']/RAW_DATA_df['params.traces']
SUMMARY['time'] = RAW_DATA_df['stats.times.total']
SUMMARY['timeout'] = RAW_DATA_df['stats.timedout']
SUMMARY['hamming'] = RAW_DATA_df['control_flow.hamming']
SUMMARY['levenshtein'] = RAW_DATA_df['control_flow.levenshtein']

SUMMARY

,src,id,generator,threshold,randomised,model,case_size,log_size,generated,coverage,time,timeout,hamming,levenshtein
0,caise2025_tabu_2024-12-06T08:21+00:00,levenshtein_caise2025_experimental_model_4000_...,PBLogGeneratorLevenshtein,0.6,True,caise2025_experimental_model,70,4000,0,0.000000,0.000000,True,NaN,NaN
1,caise2025_tabu_2024-12-06T08:21+00:00,hamming_caise2025_experimental_model_2000_030_...,PBLogGeneratorHamming,0.6,False,caise2025_experimental_model,30,2000,1,0.000500,120.139404,True,NaN,NaN
2,caise2025_tabu_2024-12-06T08:21+00:00,levenshtein_caise2025_experimental_model_0500_...,PBLogGeneratorLevenshtein,0.6,True,caise2025_experimental_model,70,500,0,0.000000,0.000000,True,NaN,NaN
3,caise2025_tabu_2024-12-06T08:21+00:00,levenshtein_caise2025_experimental_model_1000_...,PBLogGeneratorLevenshtein,0.3,False,caise2025_experimental_model,50,1000,8,0.008000,480.369058,True,0.399286,0.324286
4,caise2025_tabu_2024-12-06T08:21+00:00,levenshtein_caise2025_experimental_model_8000_...,PBLogGeneratorLevenshtein,0.3,False,caise2025_experimental_model,40,8000,7,0.000875,227.800163,True,0.373810,0.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,caise2025_base_2024-12-06T08:20+00:00,old_caise2025_experimental_model_8000_050_00,PBLogGeneratorOrig,NaN,False,caise2025_experimental_model,50,8000,8000,1.000000,0.000000,False,0.203158,0.156808
788,caise2025_base_2024-12-06T08:20+00:00,random_caise2025_experimental_model_8000_050_00,PBLogGeneratorRandom,NaN,True,caise2025_experimental_model,50,8000,0,0.000000,0.000000,True,NaN,NaN
789,caise2025_base_2024-12-06T08:20+00:00,baseline_caise2025_experimental_model_4000_060_00,PBLogGeneratorBaseline,NaN,False,caise2025_experimental_model,60,4000,4000,1.000000,0.000000,False,0.000000,0.000000
790,caise2025_base_2024-12-06T08:20+00:00,random_caise2025_experimental_model_0100_060_00,PBLogGeneratorRandom,NaN,True,caise2025_experimental_model,60,100,0,0.000000,0.000000,True,NaN,NaN


### Write summary to file

In [6]:
from datetime import datetime


SUMMARY.to_csv(OUTPUT_DIR / f'summary_{datetime.now(tz=None).strftime('%Y-%m-%dT%H%M%S')}.csv')

## Data analysis

Show results for each generator class and configuration, tables are summarised by log and case sizes 

In [7]:
from IPython.display import display

def show_table(key: str) -> None:
    for k, df in SUMMARY.sort_values(by=['generator', 'randomised', 'threshold', 'src']).groupby(by=['src', 'generator', 'randomised', 'threshold'], dropna=False):
        timeout_style = lambda _: df.pivot(index='log_size', columns='case_size', values='timeout').map(lambda x: 'background-color: pink' if x else '')
        pivot_df = df.pivot(index='log_size', columns='case_size', values=key)
        display(pivot_df.style
                .set_caption(f'{k[1]}(rnd={k[2]}, threshold={k[3]}) src: {k[0]}')
                .apply(timeout_style, axis=None))

### Coverage

In [8]:
show_table('coverage')

case_size,20,30,40,50,60,70
log_size,,,,,,
100,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
500,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
8000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.070000,0.070000,0.030000,0.020000,0.020000,0.030000
500,0.014000,0.014000,0.006000,0.004000,0.004000,0.006000
1000,0.007000,0.007000,0.003000,0.002000,0.002000,0.003000
2000,0.003500,0.003500,0.001500,0.001000,0.001000,0.001500
4000,0.001750,0.001750,0.000750,0.000500,0.000500,0.000750
8000,0.000875,0.000875,0.000375,0.000250,0.000250,0.000375


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000
500,0.004000,0.004000,0.004000,0.004000,0.004000,0.004000
1000,0.002000,0.002000,0.002000,0.002000,0.002000,0.002000
2000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000
4000,0.000500,0.000500,0.000500,0.000500,0.000500,0.000500
8000,0.000250,0.000250,0.000250,0.000250,0.000250,0.000250


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.120000,0.160000,0.170000,0.010000,0.010000,0.020000
500,0.024000,0.032000,0.034000,0.002000,0.002000,0.004000
1000,0.012000,0.016000,0.017000,0.001000,0.001000,0.002000
2000,0.006000,0.008000,0.008500,0.000500,0.000500,0.001000
4000,0.003000,0.004000,0.004250,0.000250,0.000250,0.000500
8000,0.001500,0.002000,0.002125,0.000125,0.000125,0.000250


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.030000,0.020000,0.020000,0.010000,0.010000,0.010000
500,0.006000,0.004000,0.004000,0.002000,0.002000,0.002000
1000,0.003000,0.002000,0.002000,0.001000,0.001000,0.001000
2000,0.001500,0.001000,0.001000,0.000500,0.000500,0.000500
4000,0.000750,0.000500,0.000500,0.000250,0.000250,0.000250
8000,0.000375,0.000250,0.000250,0.000125,0.000125,0.000125


...

case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000
500,0.002000,0.002000,0.002000,0.000000,0.000000,0.000000
1000,0.001000,0.001000,0.001000,0.000000,0.000000,0.000000
2000,0.000500,0.000500,0.000500,0.000000,0.000000,0.000000
4000,0.000250,0.000250,0.000250,0.000000,0.000000,0.000000
8000,0.000125,0.000125,0.000125,0.000000,0.000000,0.000000


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.050000,0.070000,0.070000,0.080000,0.080000,0.040000
500,0.010000,0.014000,0.014000,0.016000,0.016000,0.008000
1000,0.005000,0.007000,0.007000,0.008000,0.008000,0.004000
2000,0.002500,0.003500,0.003500,0.004000,0.004000,0.002000
4000,0.001250,0.001750,0.001750,0.002000,0.002000,0.001000
8000,0.000625,0.000875,0.000875,0.001000,0.001000,0.000500


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000
500,0.002000,0.002000,0.002000,0.002000,0.002000,0.002000
1000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000
2000,0.000500,0.000500,0.000500,0.000500,0.000500,0.000500
4000,0.000250,0.000250,0.000250,0.000250,0.000250,0.000250
8000,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.040000,0.050000,0.010000,0.000000,0.000000,0.000000
500,0.008000,0.010000,0.002000,0.000000,0.000000,0.000000
1000,0.004000,0.005000,0.001000,0.000000,0.000000,0.000000
2000,0.002000,0.002500,0.000500,0.000000,0.000000,0.000000
4000,0.001000,0.001250,0.000250,0.000000,0.000000,0.000000
8000,0.000500,0.000625,0.000125,0.000000,0.000000,0.000000


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.010000,0.020000,0.020000,0.000000,0.000000,0.000000
500,0.002000,0.004000,0.004000,0.000000,0.000000,0.000000
1000,0.001000,0.002000,0.002000,0.000000,0.000000,0.000000
2000,0.000500,0.001000,0.001000,0.000000,0.000000,0.000000
4000,0.000250,0.000500,0.000500,0.000000,0.000000,0.000000
8000,0.000125,0.000250,0.000250,0.000000,0.000000,0.000000


### Time

In [9]:
show_table('time')

case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.397714,0.762293,1.436808,2.028225,2.522400,4.258195
500,0.603830,1.074463,1.954592,2.454906,3.007906,4.827375
1000,0.807571,1.312318,2.154892,2.908734,3.730215,5.423185
2000,1.225278,2.072190,2.991883,3.921039,5.084914,7.006351
4000,2.179257,3.235565,4.734706,8.074038,11.094582,13.933042
8000,5.587162,7.419210,10.730083,16.502359,22.248706,21.904975


case_size,20,30,40,50,60,70
log_size,,,,,,
100,488.219286,470.626528,330.045934,243.041690,244.165053,283.046954
500,479.864117,468.698910,326.392089,243.256580,244.147620,282.110333
1000,475.527728,464.493109,328.315792,243.410357,244.521894,281.747079
2000,477.198426,466.016812,326.996696,243.338099,244.214089,287.061373
4000,492.059595,481.745667,331.309789,243.680941,244.549708,286.933052
8000,482.836862,471.830861,332.921834,243.577432,244.519087,283.641483


case_size,20,30,40,50,60,70
log_size,,,,,,
100,240.595467,241.347853,242.198254,243.208141,244.223809,246.374155
500,240.691875,241.414524,242.386213,243.329013,244.249114,246.318991
1000,240.686343,241.361799,242.416742,243.595735,244.175926,246.218887
2000,240.693917,241.398601,242.383911,243.328998,244.202319,246.742137
4000,240.840587,241.646778,242.783883,243.609472,244.716571,246.353464
8000,240.861079,241.557910,242.525229,243.479443,244.486145,246.702724


case_size,20,30,40,50,60,70
log_size,,,,,,
100,333.738789,486.837594,558.025017,128.704659,130.230732,194.735633
500,333.700208,482.247971,558.390908,128.487150,129.567833,193.771406
1000,333.319670,482.301111,559.738223,129.214999,129.529656,193.380869
2000,333.755393,480.107078,557.192941,128.585296,129.569290,197.853821
4000,337.568324,495.424126,581.611708,129.408187,130.527051,196.060800
8000,335.492582,493.186541,574.774116,128.865773,130.355410,198.331473


case_size,20,30,40,50,60,70
log_size,,,,,,
100,302.761964,243.140807,246.913907,128.659396,130.175065,134.567106
500,301.781675,243.140166,247.070279,128.474695,129.527783,133.627637
1000,301.740469,243.184634,247.127727,129.345677,129.510095,133.634772
2000,302.058249,243.186892,247.066198,128.584126,130.530166,135.261709
4000,305.258431,243.490408,247.917676,129.164890,130.601392,135.303274
8000,303.122939,243.374553,247.202757,128.838874,129.923532,135.400656


...

case_size,20,30,40,50,60,70
log_size,,,,,,
100,120.046201,120.119907,120.198674,0.000000,0.000000,0.000000
500,120.056900,120.111668,120.175847,0.000000,0.000000,0.000000
1000,120.057673,120.118396,120.171125,0.000000,0.000000,0.000000
2000,120.055101,120.107104,120.177066,0.000000,0.000000,0.000000
4000,120.053313,120.106333,120.171945,0.000000,0.000000,0.000000
8000,120.052148,120.103277,120.173670,0.000000,0.000000,0.000000


case_size,20,30,40,50,60,70
log_size,,,,,,
100,5.586485,207.647842,232.179382,505.388575,550.578201,427.362435
500,7.460083,213.332258,232.458899,492.851817,539.972500,406.179462
1000,6.899628,206.067140,235.852755,480.369058,516.746795,394.132918
2000,5.926237,206.761071,230.373814,470.294435,518.564996,399.398393
4000,5.451726,203.185310,231.493811,472.044005,522.698802,398.391027
8000,5.432345,204.555923,227.800163,471.338093,518.824265,400.985375


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.505700,5.084974,89.324082,132.510595,145.381319,160.192808
500,0.569538,4.723549,75.539089,131.399200,140.839794,156.346931
1000,0.649417,4.994633,68.154574,131.139761,140.085559,154.814592
2000,0.571432,4.645504,78.804741,131.285761,140.025266,156.168617
4000,0.559097,4.439438,63.937493,131.081860,140.014138,155.452266
8000,0.484165,4.064394,60.962023,131.481532,140.811912,154.799807


case_size,20,30,40,50,60,70
log_size,,,,,,
100,7.650831,269.772910,125.156123,0.000000,0.000000,0.000000
500,10.686880,265.715248,124.997444,0.000000,0.000000,0.000000
1000,9.064670,266.349887,125.030462,0.000000,0.000000,0.000000
2000,7.600885,262.735542,124.865217,0.000000,0.000000,0.000000
4000,7.402617,247.965962,124.934215,0.000000,0.000000,0.000000
8000,6.639725,246.622412,124.670491,0.000000,0.000000,0.000000


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.827804,19.751400,206.265635,0.000000,0.000000,0.000000
500,0.991937,16.147863,195.832513,0.000000,0.000000,0.000000
1000,1.109639,15.636494,193.958240,0.000000,0.000000,0.000000
2000,0.877826,17.882296,194.759466,0.000000,0.000000,0.000000
4000,0.840558,16.614997,188.271911,0.000000,0.000000,0.000000
8000,0.799722,14.201742,187.911090,0.000000,0.000000,0.000000


### Levenshtein distance

In [10]:
show_table('levenshtein')

case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.176768,0.168350,0.151515,0.171717,0.185185,0.173160
500,0.175351,0.167001,0.150301,0.170341,0.183701,0.171772
1000,0.175175,0.166834,0.150150,0.170170,0.183517,0.171600
2000,0.175088,0.166750,0.150075,0.170085,0.183425,0.171514
4000,0.175044,0.166708,0.150038,0.170043,0.183379,0.171471
8000,0.175022,0.166688,0.150019,0.170021,0.183356,0.171450


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.309524,0.312698,0.291667,0.340000,0.366667,0.300000
500,0.309524,0.312698,0.291667,0.340000,0.366667,0.300000
1000,0.309524,0.312698,0.291667,0.340000,0.366667,0.300000
2000,0.309524,0.312698,0.291667,0.340000,0.366667,0.300000
4000,0.309524,0.312698,0.291667,0.340000,0.366667,0.300000
8000,0.309524,0.312698,0.291667,0.340000,0.366667,0.300000


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.350000,0.333333,0.300000,0.340000,0.366667,0.342857
500,0.350000,0.333333,0.300000,0.340000,0.366667,0.342857
1000,0.350000,0.333333,0.300000,0.340000,0.366667,0.342857
2000,0.350000,0.333333,0.300000,0.340000,0.366667,0.342857
4000,0.350000,0.333333,0.300000,0.340000,0.366667,0.342857
8000,0.350000,0.333333,0.300000,0.340000,0.366667,0.342857


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.315909,0.295278,0.292647,nan,nan,0.242857
500,0.315909,0.295278,0.292647,nan,nan,0.242857
1000,0.315909,0.295278,0.292647,nan,nan,0.242857
2000,0.315909,0.295278,0.292647,nan,nan,0.242857
4000,0.315909,0.295278,0.292647,nan,nan,0.242857
8000,0.315909,0.295278,0.292647,nan,nan,0.242857


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.433333,0.166667,0.125000,nan,nan,nan
500,0.433333,0.166667,0.125000,nan,nan,nan
1000,0.433333,0.166667,0.125000,nan,nan,nan
2000,0.433333,0.166667,0.125000,nan,nan,nan
4000,0.433333,0.166667,0.125000,nan,nan,nan
8000,0.433333,0.166667,0.125000,nan,nan,nan


...

case_size,20,30,40,50,60,70
log_size,,,,,,
100,nan,nan,nan,nan,nan,nan
500,nan,nan,nan,nan,nan,nan
1000,nan,nan,nan,nan,nan,nan
2000,nan,nan,nan,nan,nan,nan
4000,nan,nan,nan,nan,nan,nan
8000,nan,nan,nan,nan,nan,nan


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.300000,0.315873,0.316667,0.324286,0.330357,0.311905
500,0.300000,0.315873,0.316667,0.324286,0.330357,0.311905
1000,0.300000,0.315873,0.316667,0.324286,0.330357,0.311905
2000,0.300000,0.315873,0.316667,0.324286,0.330357,0.311905
4000,0.300000,0.315873,0.316667,0.324286,0.330357,0.311905
8000,0.300000,0.315873,0.316667,0.324286,0.330357,0.311905


case_size,20,30,40,50,60,70
log_size,,,,,,
100,nan,nan,nan,nan,nan,nan
500,nan,nan,nan,nan,nan,nan
1000,nan,nan,nan,nan,nan,nan
2000,nan,nan,nan,nan,nan,nan
4000,nan,nan,nan,nan,nan,nan
8000,nan,nan,nan,nan,nan,nan


case_size,20,30,40,50,60,70
log_size,,,,,,
100,0.316667,0.370000,nan,nan,nan,nan
500,0.316667,0.370000,nan,nan,nan,nan
1000,0.316667,0.370000,nan,nan,nan,nan
2000,0.316667,0.370000,nan,nan,nan,nan
4000,0.316667,0.370000,nan,nan,nan,nan
8000,0.316667,0.370000,nan,nan,nan,nan


case_size,20,30,40,50,60,70
log_size,,,,,,
100,nan,0.600000,0.600000,nan,nan,nan
500,nan,0.600000,0.600000,nan,nan,nan
1000,nan,0.600000,0.600000,nan,nan,nan
2000,nan,0.600000,0.600000,nan,nan,nan
4000,nan,0.600000,0.600000,nan,nan,nan
8000,nan,0.600000,0.600000,nan,nan,nan


### Generated log size

In [11]:
show_table('generated')

case_size,20,30,40,50,60,70
log_size,,,,,,
100,100,100,100,100,100,100
500,500,500,500,500,500,500
1000,1000,1000,1000,1000,1000,1000
2000,2000,2000,2000,2000,2000,2000
4000,4000,4000,4000,4000,4000,4000
8000,8000,8000,8000,8000,8000,8000


case_size,20,30,40,50,60,70
log_size,,,,,,
100,7,7,3,2,2,3
500,7,7,3,2,2,3
1000,7,7,3,2,2,3
2000,7,7,3,2,2,3
4000,7,7,3,2,2,3
8000,7,7,3,2,2,3


case_size,20,30,40,50,60,70
log_size,,,,,,
100,2,2,2,2,2,2
500,2,2,2,2,2,2
1000,2,2,2,2,2,2
2000,2,2,2,2,2,2
4000,2,2,2,2,2,2
8000,2,2,2,2,2,2


case_size,20,30,40,50,60,70
log_size,,,,,,
100,12,16,17,1,1,2
500,12,16,17,1,1,2
1000,12,16,17,1,1,2
2000,12,16,17,1,1,2
4000,12,16,17,1,1,2
8000,12,16,17,1,1,2


case_size,20,30,40,50,60,70
log_size,,,,,,
100,3,2,2,1,1,1
500,3,2,2,1,1,1
1000,3,2,2,1,1,1
2000,3,2,2,1,1,1
4000,3,2,2,1,1,1
8000,3,2,2,1,1,1


...

case_size,20,30,40,50,60,70
log_size,,,,,,
100,1,1,1,0,0,0
500,1,1,1,0,0,0
1000,1,1,1,0,0,0
2000,1,1,1,0,0,0
4000,1,1,1,0,0,0
8000,1,1,1,0,0,0


case_size,20,30,40,50,60,70
log_size,,,,,,
100,5,7,7,8,8,4
500,5,7,7,8,8,4
1000,5,7,7,8,8,4
2000,5,7,7,8,8,4
4000,5,7,7,8,8,4
8000,5,7,7,8,8,4


case_size,20,30,40,50,60,70
log_size,,,,,,
100,1,1,1,1,1,1
500,1,1,1,1,1,1
1000,1,1,1,1,1,1
2000,1,1,1,1,1,1
4000,1,1,1,1,1,1
8000,1,1,1,1,1,1


case_size,20,30,40,50,60,70
log_size,,,,,,
100,4,5,1,0,0,0
500,4,5,1,0,0,0
1000,4,5,1,0,0,0
2000,4,5,1,0,0,0
4000,4,5,1,0,0,0
8000,4,5,1,0,0,0


case_size,20,30,40,50,60,70
log_size,,,,,,
100,1,2,2,0,0,0
500,1,2,2,0,0,0
1000,1,2,2,0,0,0
2000,1,2,2,0,0,0
4000,1,2,2,0,0,0
8000,1,2,2,0,0,0


### Timed out

In [12]:
show_table('timeout')

case_size,20,30,40,50,60,70
log_size,,,,,,
100,False,False,False,False,False,False
500,False,False,False,False,False,False
1000,False,False,False,False,False,False
2000,False,False,False,False,False,False
4000,False,False,False,False,False,False
8000,False,False,False,False,False,False


case_size,20,30,40,50,60,70
log_size,,,,,,
100,True,True,True,True,True,True
500,True,True,True,True,True,True
1000,True,True,True,True,True,True
2000,True,True,True,True,True,True
4000,True,True,True,True,True,True
8000,True,True,True,True,True,True


case_size,20,30,40,50,60,70
log_size,,,,,,
100,True,True,True,True,True,True
500,True,True,True,True,True,True
1000,True,True,True,True,True,True
2000,True,True,True,True,True,True
4000,True,True,True,True,True,True
8000,True,True,True,True,True,True


case_size,20,30,40,50,60,70
log_size,,,,,,
100,True,True,True,True,True,True
500,True,True,True,True,True,True
1000,True,True,True,True,True,True
2000,True,True,True,True,True,True
4000,True,True,True,True,True,True
8000,True,True,True,True,True,True


case_size,20,30,40,50,60,70
log_size,,,,,,
100,True,True,True,True,True,True
500,True,True,True,True,True,True
1000,True,True,True,True,True,True
2000,True,True,True,True,True,True
4000,True,True,True,True,True,True
8000,True,True,True,True,True,True


...

case_size,20,30,40,50,60,70
log_size,,,,,,
100,True,True,True,True,True,True
500,True,True,True,True,True,True
1000,True,True,True,True,True,True
2000,True,True,True,True,True,True
4000,True,True,True,True,True,True
8000,True,True,True,True,True,True


case_size,20,30,40,50,60,70
log_size,,,,,,
100,False,True,True,True,True,True
500,False,True,True,True,True,True
1000,False,True,True,True,True,True
2000,False,True,True,True,True,True
4000,False,True,True,True,True,True
8000,False,True,True,True,True,True


case_size,20,30,40,50,60,70
log_size,,,,,,
100,False,False,False,True,True,True
500,False,False,False,True,True,True
1000,False,False,False,True,True,True
2000,False,False,False,True,True,True
4000,False,False,False,True,True,True
8000,False,False,False,True,True,True


case_size,20,30,40,50,60,70
log_size,,,,,,
100,False,True,True,True,True,True
500,False,True,True,True,True,True
1000,False,True,True,True,True,True
2000,False,True,True,True,True,True
4000,False,True,True,True,True,True
8000,False,True,True,True,True,True


case_size,20,30,40,50,60,70
log_size,,,,,,
100,False,False,True,True,True,True
500,False,False,True,True,True,True
1000,False,False,True,True,True,True
2000,False,False,True,True,True,True
4000,False,False,True,True,True,True
8000,False,False,True,True,True,True
